In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Load the diabetes dataset to a pandas DataFrame
diabetes_dataset = pd.read_csv('diabetes.csv')

# Print the first 5 rows of the dataset
print(diabetes_dataset.head())

# Get the number of rows and columns in this dataset
print(diabetes_dataset.shape)

# Get the statistical measures of the data
print(diabetes_dataset.describe())

# Get the distribution of the Outcome
print(diabetes_dataset['Outcome'].value_counts())

# Group by Outcome and calculate mean for each group
print(diabetes_dataset.groupby('Outcome').mean())


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
(768, 9)
       Pregnancies     Glucose  BloodPressure  SkinThickness     Insulin  \
count   768.000000  768.000000     768.000000     768.000000  768.000000   
mean      3.845052  120.894531      69.105469      20.536458   79.799479   
std       3.369578   31.972618      19.355807      15.952218  115.

In [ ]:
# Separate the data and labels
X = diabetes_dataset.drop(columns='Outcome', axis=1)
Y = diabetes_dataset['Outcome']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Define the DNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train, epochs=100, validation_split=0.2, batch_size=32)


Epoch 1/100
16/16 [==============================] - 4s 47ms/step - loss: 0.6733 - accuracy: 0.6354 - val_loss: 0.6505 - val_accuracy: 0.5935
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6433 - accuracy: 0.6762 - val_loss: 0.6326 - val_accuracy: 0.6504
Epoch 3/100
16/16 [==============================] - 0s 6ms/step - loss: 0.6207 - accuracy: 0.7026 - val_loss: 0.6129 - val_accuracy: 0.6667
Epoch 4/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5973 - accuracy: 0.7291 - val_loss: 0.5953 - val_accuracy: 0.6748
Epoch 5/100
16/16 [==============================] - 0s 4ms/step - loss: 0.5756 - accuracy: 0.7413 - val_loss: 0.5785 - val_accuracy: 0.7073
Epoch 6/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5553 - accuracy: 0.7597 - val_loss: 0.5645 - val_accuracy: 0.7317
Epoch 7/100
16/16 [==============================] - 0s 5ms/step - loss: 0.5368 - accuracy: 0.7719 - val_loss: 0.5509 - val_accuracy: 0.7317
Epoch 8/100


In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, Y_test)
print('Test accuracy:', test_accuracy)

# Predict on the test data
Y_pred = (model.predict(X_test) > 0.5).astype("int32")
print('Accuracy score on test data:', accuracy_score(Y_test, Y_pred))

5/5 [==============================] - 0s 6ms/step - loss: 0.5398 - accuracy: 0.7403
Test accuracy: 0.7402597665786743
5/5 [==============================] - 0s 3ms/step
Accuracy score on test data: 0.7402597402597403


In [ ]:
def predict_diabetes(input_data):
    # Convert the input data to a numpy array
    input_data_as_numpy_array = np.asarray(input_data)
    # Reshape the array as we are predicting for one instance
    input_data_reshaped = input_data_as_numpy_array.reshape(1, -1)
    # Standardize the input data
    input_data_reshaped = scaler.transform(input_data_reshaped)
    # Make prediction
    prediction = (model.predict(input_data_reshaped) > 0.5).astype("int32")
    if prediction[0][0] == 0:
        print('The person is not diabetic')
    else:
        print('The person is diabetic')

# Test the predictive system
input_data = (5, 166, 72, 19, 175, 25.8, 0.587, 51)
predict_diabetes(input_data)


1/1 [==============================] - 0s 40ms/step
The person is diabetic


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
# Save the model
model.save('diabetes_dnn_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('diabetes_dnn_model.h5')

# Test the loaded model
predict_diabetes(input_data)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 59ms/step
The person is diabetic
